In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportions_chisquare

## Bring in data

In [4]:
NYR=pd.read_csv("C:/Users/Repai/Downloads/New_years_resolutions.csv")

## Print first 5 columns

In [ ]:
NYR.head()

,tweet_created,tweet_text,tweet_category,tweet_topics,tweet_location,tweet_state,tweet_region,user_timezone,user_gender,retweet_count
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,"City of Angels, CA",CA,West,Pacific Time (US & Canada),male,NaN
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,"Arlington, TX",TX,South,Central Time (US & Canada),male,NaN
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,"Las Vegas, NV",NV,West,Central Time (US & Canada),male,NaN
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,"Ft. Luderdale, Florida",FL,South,NaN,male,NaN
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,South Carolina,SC,South,Pacific Time (US & Canada),female,NaN


## Renaming Columns

In [10]:
NYR.rename(columns={'tweet_created' : 'Date', 'tweet_text' : 'Tweet', 'tweet_category' : 'Category', 'tweet_topics' : 'Topics', 'tweet_location' : 'Location', 'tweet_state' : 'State', 'tweet_region' : 'Region', 'user_timezone' : 'TZ', 'user_gender' : 'Gender'}, inplace=True)

In [11]:
NYR.head()

,Date,Tweet,Category,Topics,Location,State,Region,TZ,Gender,retweet_count
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,"City of Angels, CA",CA,West,Pacific Time (US & Canada),male,NaN
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,"Arlington, TX",TX,South,Central Time (US & Canada),male,NaN
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,"Las Vegas, NV",NV,West,Central Time (US & Canada),male,NaN
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,"Ft. Luderdale, Florida",FL,South,NaN,male,NaN
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,South Carolina,SC,South,Pacific Time (US & Canada),female,NaN


## Merge Location and State

In [12]:
NYR['City and State'] = NYR["Location"] + " " + NYR["State"].map(str)

In [13]:
NYR.head()

,Date,Tweet,Category,Topics,Location,State,Region,TZ,Gender,retweet_count,City and State
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,"City of Angels, CA",CA,West,Pacific Time (US & Canada),male,NaN,"City of Angels, CA CA"
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,"Arlington, TX",TX,South,Central Time (US & Canada),male,NaN,"Arlington, TX TX"
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,"Las Vegas, NV",NV,West,Central Time (US & Canada),male,NaN,"Las Vegas, NV NV"
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,"Ft. Luderdale, Florida",FL,South,NaN,male,NaN,"Ft. Luderdale, Florida FL"
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,South Carolina,SC,South,Pacific Time (US & Canada),female,NaN,South Carolina SC


## Drop Columns

In [18]:
NYRtrim = NYR.drop(['Location', 'State', 'retweet_count'], axis=1)

## Recode

In [19]:
recodeDict = {"Gender": {"male": 1, "female": 0},
          "Region": {"North": 1, "East": 2, "South": 3, "West" : 4}}           
NYRtrim.replace(recodeDict, inplace=True)

In [20]:
NYRtrim.head()

,Date,Tweet,Category,Topics,Region,TZ,Gender,City and State
0,2014-12-21 16:11:00,#NewYearsResolution to not put the parking lot...,Humor,Humor about Personal Growth and Interests Reso...,4,Pacific Time (US & Canada),1,"City of Angels, CA CA"
1,2014-12-21 16:16:00,I got the hair. I just need to work on the bod...,Health & Fitness,Improve my body,3,Central Time (US & Canada),1,"Arlington, TX TX"
2,2014-12-21 16:30:00,"Self improvement!.. Mentally, physically, and ...",Personal Growth,Improve my attitude,4,Central Time (US & Canada),1,"Las Vegas, NV NV"
3,2014-12-21 17:51:00,I've decided to get pregnant in #2015 #NewYear...,Humor,Humor about Family/Friends/Relationships Resol...,3,NaN,1,"Ft. Luderdale, Florida FL"
4,2014-12-21 18:09:00,All I want for Christmas is to be Carrie Under...,Personal Growth,Other,3,Pacific Time (US & Canada),0,South Carolina SC
